# Chatting with a PDF

## Python requirements

In [77]:
%pip install langchain_ollama
%pip install langchain_core
%pip install langchain_text_splitters
%pip install langchain_community
%pip install unstructured
%pip install PyPDF2


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgr

## Setup some imports

In [78]:
# Imports
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Read the PDF into memory

In [79]:
from PyPDF2 import PdfReader

# Load PDF
local_path = "penta.pdf"
if local_path:
    reader = PdfReader(local_path)
    data = [page.extract_text() for page in reader.pages]
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: penta.pdf


## Split the PDF

In [80]:
from langchain.docstore.document import Document

# Wrap each page's text in a Document object
documents = [Document(page_content=page) for page in data]

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)  # Use the wrapped documents
print(f"Text split into {len(chunks)} chunks")

Text split into 76 chunks


## Pull some models

In [81]:
!ollama pull nomic-embed-text
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B         

## Store the pages in a vector DB

In [82]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Setup the langchain

In [83]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def chat(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

## Ask some questions

In [84]:
# Example 1
chat("What is the main idea of this document?")

The main idea of this document appears to be an explanation and breakdown of a simplified model called PENTA, which is used to describe and analyze 5 basic conflicting forces in any system. The PENTA model consists of Scope, Values, Efficiency, Resources, and Designs, and each component is further explained with its own definitions and examples.

In [89]:
# Example 2
chat("How can we get start implementing the penta model? be explicit in the steps.")

Based on the provided context, here are the explicit steps to get started with implementing the Penta Model:

**Step 1: Identify and Document Current Scope**

* Determine the specified set of stakeholder- and system-functions (what it must do) and constraints (what it must not do).
* Draw a border around the given system.
* Write down the current scope in detail.

**Step 2: Define Values**

* Identify the specified set of stakeholder values ("wants", "needs", "wishes", "visions") and system qualities, including system performance attributes ("potential values" for stakeholders).
* Document these values clearly and concisely.

**Step 3: Assess Efficiency**

* Calculate the current effectiveness-to-costs ratio.
* Include all stakeholder-values actually delivered in the calculation.
* Note down the life-cycle costs (not just "capital" costs).

**Step 4: Prioritize Resources**

* Identify any critical and prioritized set of limited resources for the system lifecycle (e.g., time, money, people, space).
* Document these resources clearly.

**Step 5: Analyze Interconnectedness**

* Consider how changes to one Penta requirement could influence other requirements (positively, negatively, or dynamically unpredictable ways).
* Identify potential trade-offs and areas of interconnectedness.

**Step 6: Determine Priorities**

* Decide on the priority level for each Penta requirement based on stakeholder values and system performance attributes.
* Consider adjusting other Penta requirements or Designs to support the prioritization.

**Step 7: Create a Clear and Detailed Penta Requirement**

* Write down the Penta requirement in clear and concise language using Planguage [CE] logic and specification.
* Ensure that the requirement is well-defined and easy to understand.

**Step 8: Review and Refine**

* Review the Penta model and requirements for clarity and consistency.
* Refine the model as needed to ensure it accurately represents the system's needs and constraints.

In [86]:
# Example 3
chat("Are there any contradictions or problems with the penta model?")

There are no explicit contradictions or problems mentioned in the provided context. However, it is acknowledged that "there are some real limits to what is actually possible in the real world for products, services and process efficiency" (Reality Limits). This implies that there may be limitations to the effectiveness of the Penta model in achieving optimal balance.

Additionally, it is noted that changing one Penta requirement can have unpredictable effects on other requirements, both positive and negative (Interconnectedness), which could potentially lead to problems if not managed carefully.

However, these limitations are framed as challenges to be addressed through the use of the Penta model rather than as inherent contradictions or problems with the model itself.

In [87]:
# Example 3
chat("What problems might the penta model cause in my organisation?")

Based on the provided context, it appears that the Penta Model might not directly "cause" problems in an organization, but rather provide a framework for managing risks and conflicts. However, some potential issues that could arise from implementing or relying heavily on the Penta Model include:

1. Overemphasis on analysis: The Penta Model's focus on explicit connection of all related components, synergistic planning, and incremental change might lead to an overemphasis on analysis at the expense of practical implementation.
2. Complexity: The model's complexity and use of specialized terminology (e.g., "Penta Forces," "Planguage") might create barriers for non-technical stakeholders or team members who are not familiar with these concepts.
3. Over-engineering: The emphasis on systems engineering perspective, numerical specification of values and resources, and numeric engineering logic might lead to over-engineering or analysis paralysis, where teams become too focused on optimizing every aspect of the system rather than delivering a viable product.
4. Inflexibility: The Penta Model's structure and constraints (e.g., the five Quints, explicit incremental change) might make it difficult for teams to adapt to changing requirements or unexpected problems that arise during development.
5. Dependence on data quality: The model's reliance on accurate measurement of values and resources, as well as continuous feedback adjustment, requires high-quality data to be effective. Poor data quality or inadequate feedback mechanisms might hinder the model's ability to provide useful insights.

It is essential to note that these potential issues can be mitigated by carefully selecting team members, providing adequate training and support, and ensuring that the Penta Model is used in conjunction with other complementary methodologies and tools.

In [92]:
chat("Do you have any ideas on how to expand the penta model? Give three.")

Based solely on the provided context, here are three potential ideas for expanding the Penta Model:

1. **Incorporate Emotional and Social Aspects**: The current Penta Model focuses on Values, Resources, Designs, and interconnectedness. Expanding it to include emotional and social aspects could provide a more comprehensive understanding of innovation and stakeholder values. This could involve incorporating variables such as emotional well-being, social impact, or community engagement.

2. **Integrate Machine Learning and Predictive Analytics**: The Penta Model uses logic and specification to inform tradeoff decisions and prioritize requirements. Expanding it to incorporate machine learning algorithms and predictive analytics could enable the model to better anticipate and mitigate potential issues, or identify new opportunities for innovation.

3. **Encompassing Contextual Factors**: The current Penta Model assumes a fixed set of requirements and designs. Expanding it to account for contextual factors such as environmental sustainability, cultural sensitivity, or regulatory compliance could provide a more nuanced understanding of the complex interactions within systems. This could involve incorporating variables such as environmental impact or social responsibility into the model.

These ideas are speculative and based solely on the provided context. A more in-depth analysis of the Penta Model and its underlying principles would be necessary to determine their feasibility and potential for expansion.